In [1]:
import pandas as pd

In [2]:
trafficCrashesDf = pd.read_csv('traffic-crashes.csv')
trafficCrashesDf

FileNotFoundError: [Errno 2] No such file or directory: 'traffic-crashes.csv'

In [ ]:
trafficCrashesDf.describe()

In [ ]:
trafficCrashesDf.head()

In [ ]:
trafficCrashesDf.nunique()

In [ ]:
trafficCrashesDf.isnull().sum()

In [ ]:
trafficCrashesDf.dtypes

In [ ]:
columns_to_drop =['PHOTOS_TAKEN_I','STATEMENTS_TAKEN_I','WORK_ZONE_TYPE','WORKERS_PRESENT_I','INJURIES_FATAL','INJURIES_INCAPACITATING','INJURIES_NON_INCAPACITATING','INJURIES_REPORTED_NOT_EVIDENT','INJURIES_NO_INDICATION','INJURIES_UNKNOWN','LATITUDE','LONGITUDE','LOCATION']
trafficCrashesDf.drop(columns_to_drop,axis=1,inplace=True)
trafficCrashesDf

# randomly fill nan
columns: `CRASH_DATE_EST_I` * `LANE_CNT` * `INTERSECTION_RELATED_I` * `NOT_RIGHT_OF_WAY_I` * `HIT_AND_RUN_I` * `DOORING_I` * `WORK_ZONE_I` 

In [ ]:
import numpy as np
# trafficCrashesDf1 = trafficCrashesDf.copy()
uniqueCategories = trafficCrashesDf['CRASH_DATE_EST_I'].dropna().unique()
uniqueCategories

In [ ]:
def random(column):
    np.random.seed(42)  # For reproducibility
    trafficCrashesDf[column] = trafficCrashesDf[column].apply(
    lambda x: np.random.choice(uniqueCategories) if pd.isna(x) else x
    )   

In [ ]:
random('CRASH_DATE_EST_I')
random('LANE_CNT')
random('INTERSECTION_RELATED_I')
random('NOT_RIGHT_OF_WAY_I')
random('HIT_AND_RUN_I')
random('DOORING_I')
random('WORK_ZONE_I')

In [ ]:
trafficCrashesDf

# Mode 
columns: `REPORT_TYPE` * `STREET_DIRECTION` * `STREET_NAME` * `MOST_SEVERE_INJURY`

# Mean 
columns: `BEAT_OF_OCCURRENCE` * `INJURIES_TOTAL` 

In [ ]:
def checkValues(column):
    uniqueCategories = trafficCrashesDf[column].unique()
    nullCount= trafficCrashesDf[column].isnull().sum()
    print('unique Categories: ',uniqueCategories,'\nnull count: ', nullCount)
    print('Mode:\n',trafficCrashesDf[column].value_counts())
    return trafficCrashesDf[column].mode()[0]

In [ ]:
# REPORT_TYPE
uniqueCategoriesReportType = checkValues('REPORT_TYPE')

In [ ]:
trafficCrashesDf['REPORT_TYPE'].fillna(uniqueCategoriesReportType, inplace=True)

In [ ]:
# STREET_DIRECTION
uniqueCategoriesStreetDirection = checkValues('STREET_DIRECTION')

In [ ]:
trafficCrashesDf['REPORT_TYPE'].fillna(uniqueCategoriesStreetDirection, inplace=True)


In [ ]:
# STREET_NAME
uniqueCategoriesStreetName = checkValues('STREET_NAME')

In [ ]:
trafficCrashesDf['STREET_NAME'].fillna(uniqueCategoriesStreetName,inplace=True)

In [ ]:
# MOST_SEVERE_INJURY
uniqueCategoriesMostSevereInjury = checkValues('MOST_SEVERE_INJURY')

In [ ]:
trafficCrashesDf['MOST_SEVERE_INJURY'].fillna(uniqueCategoriesMostSevereInjury, inplace=True)

In [ ]:
def calculateMean(column):
    return trafficCrashesDf[column].mean()

In [ ]:
# BEAT_OF_OCCURRENCE`
meanBeatOfOccurrence = calculateMean('BEAT_OF_OCCURRENCE')
trafficCrashesDf['BEAT_OF_OCCURRENCE'].fillna(meanBeatOfOccurrence, inplace=True)

In [ ]:
# INJURIES_TOTAL
meanInjuriesTotal = calculateMean('INJURIES_TOTAL')
trafficCrashesDf['INJURIES_TOTAL'].fillna(meanInjuriesTotal, inplace=True)

In [ ]:
trafficCrashesDf.to_csv('traffic-crashes-normal-fill.csv',index=False)

# KNN

In [ ]:
trafficCrashesDf1 = trafficCrashesDf.filter(items=['CRASH_DATE_EST_I','CRASH_DATE','DATE_POLICE_NOTIFIED']).copy()
trafficCrashesDf1

,CRASH_DATE_EST_I,CRASH_DATE,DATE_POLICE_NOTIFIED
0,NaN,08/18/2023 12:50:00 PM,08/18/2023 12:55:00 PM
1,NaN,07/29/2023 02:45:00 PM,07/29/2023 02:45:00 PM
2,NaN,08/18/2023 05:58:00 PM,08/18/2023 06:01:00 PM
3,NaN,11/26/2019 08:38:00 AM,11/26/2019 08:38:00 AM
4,NaN,08/18/2023 10:45:00 AM,08/18/2023 10:48:00 AM
...,...,...,...
823952,NaN,07/18/2023 02:10:00 PM,07/18/2023 04:55:00 PM
823953,NaN,08/05/2023 01:00:00 PM,08/05/2023 02:50:00 PM
823954,NaN,11/05/2021 10:56:00 PM,11/06/2021 12:18:00 AM
823955,NaN,10/18/2020 12:03:00 AM,10/18/2020 12:03:00 AM


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Convert categorical column to numeric values
le = LabelEncoder()
trafficCrashesDf1['Category_encoded'] = le.fit_transform(trafficCrashesDf['CRASH_DATE_EST_I'].astype(str))
trafficCrashesDf1['CRASH_DATE'] = pd.to_datetime(trafficCrashesDf1['CRASH_DATE'])
trafficCrashesDf1['DATE_POLICE_NOTIFIED'] = pd.to_datetime(trafficCrashesDf1['DATE_POLICE_NOTIFIED'])
trafficCrashesDf1['CRASH_DATE'] = trafficCrashesDf1['CRASH_DATE'].view('int64') / 10**9
trafficCrashesDf1['DATE_POLICE_NOTIFIED'] = trafficCrashesDf1['DATE_POLICE_NOTIFIED'].view('int64') / 10**9


In [ ]:
from sklearn.impute import KNNImputer

# KNN Imputation on the entire DataFrame
imputer = KNNImputer(n_neighbors=2)
df_imputed = trafficCrashesDf1.copy()
df_imputed[['CRASH_DATE', 'DATE_POLICE_NOTIFIED', 'Category_encoded']] = imputer.fit_transform(trafficCrashesDf1[['CRASH_DATE', 'DATE_POLICE_NOTIFIED', 'Category_encoded']])


In [ ]:
# Convert the encoded categories back to the original categorical values
df_imputed['Category'] = le.inverse_transform(df_imputed['Category_encoded'].astype(int))
df_imputed['CRASH_DATE'] = pd.to_datetime(df_imputed['CRASH_DATE'], unit='s')
df_imputed['DATE_POLICE_NOTIFIED'] = pd.to_datetime(df_imputed['DATE_POLICE_NOTIFIED'], unit='s')

# Drop the temporary encoded column
df_imputed = df_imputed.drop(columns=['Category_encoded'])

print("\nDataFrame after KNN imputation:")
print(df_imputed)


DataFrame after KNN imputation:
       CRASH_DATE_EST_I          CRASH_DATE DATE_POLICE_NOTIFIED Category
0                   NaN 2023-08-18 12:50:00  2023-08-18 12:55:00      nan
1                   NaN 2023-07-29 14:45:00  2023-07-29 14:45:00      nan
2                   NaN 2023-08-18 17:58:00  2023-08-18 18:01:00      nan
3                   NaN 2019-11-26 08:38:00  2019-11-26 08:38:00      nan
4                   NaN 2023-08-18 10:45:00  2023-08-18 10:48:00      nan
...                 ...                 ...                  ...      ...
823952              NaN 2023-07-18 14:10:00  2023-07-18 16:55:00      nan
823953              NaN 2023-08-05 13:00:00  2023-08-05 14:50:00      nan
823954              NaN 2021-11-05 22:56:00  2021-11-06 00:18:00      nan
823955              NaN 2020-10-18 00:03:00  2020-10-18 00:03:00      nan
823956              NaN 2022-03-15 02:00:00  2022-03-15 14:00:00      nan

[823957 rows x 4 columns]


In [ ]:
# import pandas as pd
# from sklearn.impute import KNNImputer
# import numpy as np

# trafficCrashesDf1 = trafficCrashesDf.filter(items=['CRASH_DATE_EST_I','CRASH_DATE','DATE_POLICE_NOTIFIED']).copy()

# # Convert datetime columns to pandas datetime
# trafficCrashesDf1['CRASH_DATE'] = pd.to_datetime(trafficCrashesDf1['CRASH_DATE'])
# trafficCrashesDf1['DATE_POLICE_NOTIFIED'] = pd.to_datetime(trafficCrashesDf1['DATE_POLICE_NOTIFIED'])

# # Convert datetime columns to Unix timestamps
# trafficCrashesDf1['CRASH_DATE'] = trafficCrashesDf1['CRASH_DATE'].view('int64') / 10**9
# trafficCrashesDf1['DATE_POLICE_NOTIFIED'] = trafficCrashesDf1['DATE_POLICE_NOTIFIED'].view('int64') / 10**9

# # Initialize the imputer
# imputer = KNNImputer(n_neighbors=2)

# # Apply imputer on the relevant columns
# imputed_data = imputer.fit_transform(trafficCrashesDf1[['CRASH_DATE', 'DATE_POLICE_NOTIFIED', 'CRASH_DATE_EST_I']])

# # Convert back to DataFrame
# df_imputed = pd.DataFrame(imputed_data, columns=['CRASH_DATE', 'DATE_POLICE_NOTIFIED', 'CRASH_DATE_EST_I'])

# # Convert numerical datetime back to original datetime format
# df_imputed['CRASH_DATE'] = pd.to_datetime(df_imputed['CRASH_DATE'], unit='s')
# df_imputed['DATE_POLICE_NOTIFIED'] = pd.to_datetime(df_imputed['DATE_POLICE_NOTIFIED'], unit='s')

# # Print the imputed DataFrame
# print(df_imputed)


ValueError: could not convert string to float: 'Y'